In [4]:
from google import genai

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works in a few words"
)
print(response.text)

AI analyzes data to learn patterns, then uses those patterns to make predictions or decisions.


This is my personal solution to hw2, we will be using mirascope for all LLM calls

In [14]:
# import and testing the api
from mirascope import llm, prompt_template
from dotenv import load_dotenv
import os
from google.genai import Client

# Load environment variables from .env file
load_dotenv()

client = Client(api_key=os.getenv('GOOGLE_API_KEY'))

@llm.call(provider="google", model="gemini-2.0-flash", client=client)
def get_capital(country: str) -> str:
    return f"What is the capital of {country}?"


response = get_capital("Japan")
print(response.content)

The capital of Japan is **Tokyo**.



## Part 1: Define dimensions and Generating Initial Queries

first we have to get the key dimensions we care about for our recipe bot
based on that we shall generate the inital queries

### Identifying key dimensions

1. **Dietary Restrictions**: What dietary limits exists?
    - halal
    - low calories
    - high protein

2. **What for**: what is the meal for?
    - to pack up and take to lunch
    - for breakfast but eat at work
    - Snacks

3. **Time Available**: How much time
    - Under 15 minutes
    - 30 minutes
    - under 30 mins

4. **Ingredient Base**:
    - Ground Beef
    - Chicken thigh/breast
    - oats

5. **Meal Time**:
   - Breakfast
   - Lunch
   - Dinner

In [26]:
from pydantic import BaseModel
from typing import Literal

class RecipeQueryDimensions(BaseModel):
    """Pydantic model representing key dimensions for recipe queries.
    
    Each field represents a dimension that affects recipe recommendations,
    using strictly typed options for data validation.
    """
    dietary_restriction: Literal["Halal", "Low-Calorie", "High-Protein", "Low Carb", "No Seafood", "Dairy-Free", "Mediterranean"]
    meal_purpose: Literal["Packed Lunch", "Breakfast at Work", "Snacks", "Dinner Party", "Cooking for the Family", "Work Lunch", "Picnic"]
    prep_time: Literal["Under 15", "30 Minutes", "1 Hour", "2 Hours", "Overnight"]
    main_ingredient: Literal["Ground Beef", "Chicken", "Oats", "Beans", "Pasta", "Tofu", "Rice", "Sweet Potatoes"]
    meal_time: Literal["Breakfast", "Lunch", "Dinner", "Brunch", "Late Night"]

In [27]:
@llm.call(provider="google", model="gemini-2.0-flash", client=client, response_model=list[RecipeQueryDimensions])
@prompt_template(
    """
    I am designing a Recipe Bot and want to test it with a diverse set of user scenarios. Please generate 50 unique combinations (tuples) using the following key dimensions and their possible values:

    Dietary Restrictions:
    - Halal, Low-Calorie, High-Protein, Low Carb, No Seafood, Dairy-Free, Mediterranean, Pakistani, Arab

    What For:
    - Packed Lunch, Breakfast at Work, Snacks, Potluck, Dinner Party, Family Cooking, Work Lunch, Picnic

    Time Available:
    - Under 15 mins, 30 Minutes, 1 Hour, 2 Hours, Overnight

    Ingredient Base:
    - Ground Beef, Chicken, Oats, Beans, Pasta, Tofu, Rice, Sweet Potatoes

    Meal Time:
    - Breakfast, Lunch, Dinner, Brunch, Late Night

    Each combination should select one value from each dimension. Present the results as a list of tuples, where each tuple contains one value for each dimension in the following order: (Dietary Restrictions, What For, Time Available, Ingredient Base, Meal Time). Ensure that the combinations are varied and realistic.
    """
)
def generate_synthetic_queries():...

In [28]:
queries = generate_synthetic_queries()
print(queries)
print(type(queries))

[RecipeQueryDimensions(dietary_restriction='Halal', meal_purpose='Packed Lunch', prep_time='Under 15', main_ingredient='Ground Beef', meal_time='Breakfast'), RecipeQueryDimensions(dietary_restriction='Low-Calorie', meal_purpose='Breakfast at Work', prep_time='30 Minutes', main_ingredient='Chicken', meal_time='Lunch'), RecipeQueryDimensions(dietary_restriction='High-Protein', meal_purpose='Snacks', prep_time='1 Hour', main_ingredient='Oats', meal_time='Dinner'), RecipeQueryDimensions(dietary_restriction='Low Carb', meal_purpose='Dinner Party', prep_time='2 Hours', main_ingredient='Beans', meal_time='Brunch'), RecipeQueryDimensions(dietary_restriction='No Seafood', meal_purpose='Cooking for the Family', prep_time='Overnight', main_ingredient='Pasta', meal_time='Late Night'), RecipeQueryDimensions(dietary_restriction='Dairy-Free', meal_purpose='Work Lunch', prep_time='Under 15', main_ingredient='Tofu', meal_time='Breakfast'), RecipeQueryDimensions(dietary_restriction='Mediterranean', meal

In [29]:
# Print each query object in a readable format with proper formatting and line breaks
for i, query in enumerate(queries, 1):
    print(f"\nQuery #{i}:")
    print(f"  Dietary Restriction: {query.dietary_restriction}")
    print(f"  Meal Purpose: {query.meal_purpose}")
    print(f"  Prep Time: {query.prep_time}")
    print(f"  Main Ingredient: {query.main_ingredient}")
    print(f"  Meal Time: {query.meal_time}")
    print("-" * 50)  # Add a separator between queries for better readability



Query #1:
  Dietary Restriction: Halal
  Meal Purpose: Packed Lunch
  Prep Time: Under 15
  Main Ingredient: Ground Beef
  Meal Time: Breakfast
--------------------------------------------------

Query #2:
  Dietary Restriction: Low-Calorie
  Meal Purpose: Breakfast at Work
  Prep Time: 30 Minutes
  Main Ingredient: Chicken
  Meal Time: Lunch
--------------------------------------------------

Query #3:
  Dietary Restriction: High-Protein
  Meal Purpose: Snacks
  Prep Time: 1 Hour
  Main Ingredient: Oats
  Meal Time: Dinner
--------------------------------------------------

Query #4:
  Dietary Restriction: Low Carb
  Meal Purpose: Dinner Party
  Prep Time: 2 Hours
  Main Ingredient: Beans
  Meal Time: Brunch
--------------------------------------------------

Query #5:
  Dietary Restriction: No Seafood
  Meal Purpose: Cooking for the Family
  Prep Time: Overnight
  Main Ingredient: Pasta
  Meal Time: Late Night
--------------------------------------------------

Query #6:
  Dietary R

In [30]:
normal_list = [tuple(item.model_dump().values()) for item in queries]

print(normal_list)

[('Halal', 'Packed Lunch', 'Under 15', 'Ground Beef', 'Breakfast'), ('Low-Calorie', 'Breakfast at Work', '30 Minutes', 'Chicken', 'Lunch'), ('High-Protein', 'Snacks', '1 Hour', 'Oats', 'Dinner'), ('Low Carb', 'Dinner Party', '2 Hours', 'Beans', 'Brunch'), ('No Seafood', 'Cooking for the Family', 'Overnight', 'Pasta', 'Late Night'), ('Dairy-Free', 'Work Lunch', 'Under 15', 'Tofu', 'Breakfast'), ('Mediterranean', 'Picnic', '30 Minutes', 'Rice', 'Lunch'), ('Halal', 'Packed Lunch', '1 Hour', 'Sweet Potatoes', 'Dinner'), ('Low-Calorie', 'Breakfast at Work', '2 Hours', 'Ground Beef', 'Brunch'), ('High-Protein', 'Snacks', 'Overnight', 'Chicken', 'Late Night'), ('Low Carb', 'Dinner Party', 'Under 15', 'Oats', 'Breakfast'), ('No Seafood', 'Cooking for the Family', '30 Minutes', 'Beans', 'Lunch'), ('Dairy-Free', 'Work Lunch', '1 Hour', 'Pasta', 'Dinner'), ('Mediterranean', 'Picnic', '2 Hours', 'Tofu', 'Brunch'), ('Halal', 'Packed Lunch', 'Overnight', 'Rice', 'Late Night'), ('Low-Calorie', 'Break

In [31]:
@llm.call(provider="google", model="gemini-2.0-flash", client=client)
@prompt_template(
    """
    Convert these dimension combinations into realistic user queries for a recipe bot. Create natural, conversational queries that reflect how real users would interact in chat interfaces like Discord or ChatGPT. Include variations in:
   - Writing style (formal vs casual)
   - Sentence structure (complete vs incomplete)
   - Common typos and informal grammar
   - Natural language patterns
   - Realistic context and urgency
   - some of queries should not include all the dimensions
   - some of queries should not include the time available
   - have some missing data in the queries
   - some queries should not have the meal time, ingredients etc. be realisitc
                         
    Include only 1 example per `dimension_example`.
                         
    <dimension_examples>
    {dimension_examples}
    </dimension_examples>
                         
   Return the results as a list of strings.""")
def get_natural_queries(dimension_examples: list):...

In [32]:
natural_queries = get_natural_queries(normal_list)
print(natural_queries)

```python
[
    "Hey, I need a halal recipe for packed lunch. Something with ground beef, maybe for tomorrow morning's breakfast, and it needs to be quick, like under 15 minutes.",
    "Can you suggest a low-calorie lunch recipe with chicken? I need it for breakfast at work in 30 minutes.",
    "Looking for high protein snacks with oats for dinner. I have an hour to prepare it.",
    "Need a low carb recipe with beans for brunch, and I have about 2 hours.",
    "Anyone got good overnight pasta ideas? No seafood for the family, late night snack.",
    "Quick dairy-free breakfast recipe? Using tofu. Gotta pack my lunch for work, under 15 mins!",
    "Help me find a Mediterranean recipe with rice for a picnic lunch? I have about 30 minutes.",
    "Halal dinner using oats - ideas?",
    "Brunch ideas, low calorie, using beef?",
    "Late night, pasta, gotta be high-protein and I can leave it overnight",
    "need a recipe, low carb, tofu, breakfast, 15 mins",
    "quick lunch recipe no sea

In [2]:
test_messages = [
        "Hey, I need a halal recipe for packed lunch. Something with ground beef, maybe for tomorrow morning's breakfast, and it needs to be quick, like under 15 minutes.",
    "Can you suggest a low-calorie lunch recipe with chicken? I need it for breakfast at work in 30 minutes.",
    "Looking for high protein snacks with oats for dinner. I have an hour to prepare it.",
    "Need a low carb recipe with beans for brunch, and I have about 2 hours.",
    "Anyone got good overnight pasta ideas? No seafood for the family, late night snack.",
    "Quick dairy-free breakfast recipe? Using tofu. Gotta pack my lunch for work, under 15 mins!",
    "Help me find a Mediterranean recipe with rice for a picnic lunch? I have about 30 minutes.",
    "Halal dinner using oats - ideas?",
    "Brunch ideas, low calorie, using beef?",
    "Late night, pasta, gotta be high-protein and I can leave it overnight",
    "need a recipe, low carb, tofu, breakfast, 15 mins",
    "quick lunch recipe no seafood using rice",
    "dinner recipe, dairy free using pasta and have one hour",
    "picnic brunch, Mediterranean with tofu, takes 2 hours",
    "Easy overnight rice recipe for late night?",
    "Low-calorie breakfast recipe, sweet potatoes, gotta be under 15 minutes!",
    "High-protein ground beef lunch recipe, I have half an hour. thx!",
    "Chicken dinner - low carb, about an hour?",
    "Looking for a no seafood brunch recipe with oats that takes 2 hours to make.",
    "Got any dairy-free late night snack recipes with beans that I can prep overnight?",
    "Quick pasta breakfast ideas, Mediterranean style. Takes less than 15?",
    "Halal packed lunch recipe with tofu, needs to be ready for lunch in 30 mins",
    "Low-calorie dinner with rice in about an hour?",
    "I'm looking for a brunch high protein recipe using sweet potatoes, I have 2 hours",
    "I need a low carb recipe using ground beef for a late night dinner party that can cook overnight",
    "Quick breakfast, no seafood, uses chicken. Got 15 mins.",
    "Easy dairy-free lunch using oats, 30 mins",
    "Mediterranean beans dinner, have an hour",
    "Halal pasta recipe for brunch. I can start making it 2 hours before.",
    "Low-calorie, overnight breakfast recipe with tofu for a late night breakfast.",
    "High protein breakfast recipe with rice. I only have 15 minutes.",
    "Sweet potato lunch recipe for low carb, 30 mins",
    "Dinner ideas with beef, no seafood, about an hour",
    "Dairy-free chicken recipe for brunch. Takes around 2 hours",
    "Mediterranean overnight oats recipe for late night",
    "Looking for a quick and easy Halal breakfast recipe with beans for a packed lunch, needs to be under 15 mins",
    "Low calorie lunch recipe with pasta, needs to be done in 30 minutes. Making it for breakfast at work.",
    "High protein dinner recipe with tofu, should take about an hour.",
    "Low carb brunch using rice, about 2 hours prep time?",
    "Overnight recipe with sweet potatoes for late night snack, no seafood?",
    "Dairy-free breakfast recipe with ground beef, must be quick (under 15 mins) for my work lunch.",
    "Mediterranean recipe with chicken for lunch? 30 minutes or less please for a picnic.",
    "Halal Dinner recipe using oats, gotta be ready in an hour for packed lunch",
    "Low-calorie brunch recipe with beans, takes 2 hours, for breakfast at work",
    "High-protein pasta recipe for late night snacks, can leave overnight.",
    "Low carb breakfast recipes with tofu ready in under 15 mins.",
    "Rice recipe for lunch, no seafood, and must take under 30 mins."
]


In [3]:
test_messages

["Hey, I need a halal recipe for packed lunch. Something with ground beef, maybe for tomorrow morning's breakfast, and it needs to be quick, like under 15 minutes.",
 'Can you suggest a low-calorie lunch recipe with chicken? I need it for breakfast at work in 30 minutes.',
 'Looking for high protein snacks with oats for dinner. I have an hour to prepare it.',
 'Need a low carb recipe with beans for brunch, and I have about 2 hours.',
 'Anyone got good overnight pasta ideas? No seafood for the family, late night snack.',
 'Quick dairy-free breakfast recipe? Using tofu. Gotta pack my lunch for work, under 15 mins!',
 'Help me find a Mediterranean recipe with rice for a picnic lunch? I have about 30 minutes.',
 'Halal dinner using oats - ideas?',
 'Brunch ideas, low calorie, using beef?',
 'Late night, pasta, gotta be high-protein and I can leave it overnight',
 'need a recipe, low carb, tofu, breakfast, 15 mins',
 'quick lunch recipe no seafood using rice',
 'dinner recipe, dairy free us

In [4]:
# Create list of IDs and write queries to CSV file
import csv

output_file = 'synthetic_queries_for_analysis_self.csv'

# Generate IDs in SYN001, SYN002, etc. format
ids = [f'SYN{str(i+1).zfill(3)}' for i in range(len(test_messages))]

# Write to CSV using built-in csv module
with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'query'])  # Write header
    writer.writerows(zip(ids, test_messages))  # Write data rows

print(f"Saved {len(test_messages)} synthetic queries to {output_file}")


Saved 47 synthetic queries to synthetic_queries_for_analysis_self.csv
